In [2]:
!pip install PyMuPDF
import fitz  # PyMuPDF
import re
import pandas as pd

# Replace 'your_pdf_file.pdf' with the path to your PDF file
pdf_file_path = '/Users/subhangidas/Desktop/Nilson.pdf'

# Open the PDF file
pdf_document = fitz.open(pdf_file_path)

# Initialize an empty string to store the extracted text
extracted_text = ""

# Iterate through each page in the PDF
for page_number in range(len(pdf_document)):
    page = pdf_document[page_number]
    
    # Extract text from the page (including text from images)
    page_text = page.get_text()
    
    # Append the page text to the extracted_text string
    extracted_text += page_text

# Close the PDF document
pdf_document.close()

# Replace line breaks with spaces to make the text continuous
continuous_text = extracted_text.replace('\n', ' ')

def extract_info(text):
    # Define a regex pattern to extract name, email, job title, and website
    pattern = r'(?:\.\s+|\n)(?P<name>[A-Z][\w\'\s]+)\s+is\s+(?P<job_title>.*?)(?:\s+at\s+(?:[\w\.]+,)?\s*(?P<email>\s*[\w\.-]+\s*@\s*[\w\.-]+\s*\.{0,2}\s*),?\s*)?\s*(?P<website>www\.[A-Za-z]+(?:\.[A-Za-z]+)?)'

    # Find all matches in the text
    matches = re.finditer(pattern, text)

    extracted_info = []

    for match in matches:
        info = match.groupdict()
        if info['name'].strip() == "www.nilsonreport.com ":
            break
        extracted_info.append(info)
        print(f"Found match: {info}")

    return extracted_info

info_list = extract_info(continuous_text)

df = pd.DataFrame(info_list)

# Specify the Excel file path where you want to save the data
excel_file = '/Users/subhangidas/Desktop/DummyOutputNilson1.xlsx'  # You can change the file name and path

# Save the DataFrame to an Excel sheet
df.to_excel(excel_file, index=False)  # Use index=False to exclude the index column

print(f'Data saved to {excel_file}')

import pandas as pd
import re

# Read the Excel file into a DataFrame
excel_file_path = '/Users/subhangidas/Desktop/DummyOutputNilson1.xlsx'
df = pd.read_excel(excel_file_path)

# Identify the index where 'www.nilsonreport.com' is present in the 'website' column
nilson_report_index = df.index[df['website'] == 'www.nilsonreport.com'].tolist()

if nilson_report_index:
    # Get the first occurrence of 'www.nilsonreport.com'
    nilson_report_index = nilson_report_index[0]

    # Trim the DataFrame to keep only the rows above 'www.nilsonreport.com'
    df = df.iloc[:nilson_report_index]

    # Function to extract email from job_title and remove it
    def extract_and_remove_email_from_job_title(job_title):
        email_matches = re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', job_title)

        # Remove trailing commas and spaces
        cleaned_job_title = job_title.rstrip(', ')

        # Remove the extracted email addresses from the job_title
        cleaned_job_title = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', cleaned_job_title)

        # Remove anything other than letters or numbers at the end
        cleaned_job_title = re.sub(r'[^A-Za-z0-9]$', '', cleaned_job_title)

        return ', '.join(email_matches), cleaned_job_title.strip()

    # Attempt to extract email addresses from job_title for rows with empty email
    rows_with_empty_email = df[df['email'].isna()]

    for index, row in rows_with_empty_email.iterrows():
        job_title = row['job_title']

        # Extract email addresses with spaces
        job_title = re.sub(r'\s*@\s*', '@', job_title)

        extracted_emails, cleaned_job_title = extract_and_remove_email_from_job_title(job_title)

        # Update the DataFrame with extracted email addresses
        if extracted_emails:
            df.at[index, 'email'] = extracted_emails

        # Update the DataFrame with cleaned job_title
        df.at[index, 'job_title'] = cleaned_job_title

    # Remove commas at the end of job descriptions
    df['job_title'] = df['job_title'].str.rstrip(',')

    # Save the updated DataFrame to a new Excel sheet
    refined_excel_file = '/Users/subhangidas/Desktop/OutputNilson1.xlsx'
    df.to_excel(refined_excel_file, index=False)

    print(f'Data with updated email addresses saved to {refined_excel_file}')
else:
    print("www.nilsonreport.com not found in the 'website' column.")


Found match: {'name': 'Erika Sanchez', 'job_title': 'VP and General Manager', 'email': 'esanchez@paypal.com', 'website': 'www.paypal.com'}
Found match: {'name': 'Craig Ramsey', 'job_title': 'Head of Real-Time Payments at ACI  Worldwide, craig.ramsey@aciworldwide.com,', 'email': None, 'website': 'www.aciworldwide.com'}
Found match: {'name': 'Tanja Haase', 'job_title': 'Head of Swift  Go', 'email': 'tanja.haase@swift.com', 'website': 'www.swift.com'}
Found match: {'name': 'Yash Kotak', 'job_title': 'Sr. Director of Product Management', 'email': 'yash.kotak@vonage.com', 'website': 'www.vonage.com'}
Found match: {'name': 'Larry Talley', 'job_title': 'CEO', 'email': 'larry@  everyware.com', 'website': 'www.everyware.com'}
Found match: {'name': 'Oded Vakrat', 'job_title': 'VP of Platform Partnerships', 'email': 'oded.vakrat@ sunbit.com', 'website': 'www.sunbit.com'}
Found match: {'name': 'Peter Stewart', 'job_title': 'EVP, North America Region', 'email': 'peter.stewart@ingenico.com', 'websit